In [ ]:
import cv2
import motor_end    # 20220401_muti_thread

camera = cv2.VideoCapture(0)
camera.set(3, 160)  #3행
camera.set(4, 120)  #4열

while camera.isOpened() :
    _, image = camera.read()
    # -1 은 180도 회전
    image = cv2.flip(image, -1)
    image = image[70:110, 30:130] # 이미지 행, 열 픽셀 슬라이싱

    #gray
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #blur
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    # 임계값인 90을 넘으면 255, 못넘으면 0으로 바꿔버림. '선만' 인식하도록.
    # 바이너리로 변형 → 선만 하얗게 인식되고, 배경은 검게 보임.
    _ , thresh = cv2.threshold(blur, 90, 255, cv2.THRESH_BINARY_INV)
    
    # 비트 연산 → 1을 0으로 → 흑백 반전
    thresh = cv2.bitwise_not(thresh)

    # erode 연산 → 침식 / 얇게 깎임
    mask = cv2.erode(thresh, None, iteration=2)
    # dilate 연산 → 두껍게
    mask = cv2.dilate(mask, None, iteration=2)
    # erode → dilate 열린 계산 / 좀 예쁘게 조정

    cv2.imshow("main", mask)
    cont, hier = cv2.findContours(mask.copy(), 1, cv2.CHAIN_APPROX_NONE)

    if len(cont) > 0 :
        # 선의 너비가 0보다 클 경우, 무게중심 찾음
        c = max(cont, key = cv2.contourArea)
        m = cv2.moments(c)
        cx = int(m["m10"] / m["m00"])
        cy = int(m["m01"] / m["m00"])
        print(cx)

        if cx >= 30 and cx <= 105 :
            print("turn right")
            motor_end.motor_one_speed(30)
            motor_end.motor_two_speed(0)
        elif cx >= 0 and cx <= 20 :
            print("turn left")
            motor_end.motor_one_speed(0)
            motor_end.motor_two_speed(30)
        else :
            print("Go")
            motor_end.motor_one_speed(30)
            motor_end.motor_two_speed(30)

    if cv2.waitKey(1) == ord("q") : #종료 명령어
        break

cv2.destroyAllWindows()
